In [ ]:
import pandas as pd
import numpy as np
import plotnine as gg
import sklearn.linear_model
import statsmodels.formula.api as smf
from plotnine import *

In [ ]:
#xs = pd.read_csv('benchmark-1490607733.csv')
xs = pd.read_csv('benchmark-1491380156.csv')
#xs = pd.read_csv('benchmark-1493248450.csv')
#xs = pd.read_csv('benchmark-1493251003.csv')

xs['config'] = xs['stage'] + '-'
for col in 'inference specialisation verification normalisation compilation'.split():
    # the comparison to `True` is a hack to set NaNs to False
    xs.loc[xs[col] == True, 'config'] += col[0]
    
xs['runtime_ms'] = 1000 * xs['runtime']

xs.head()

In [ ]:
# ttstar
ttstar = xs.loc[xs['stage'] == 'ttstar']
plot = ggplot(ttstar, aes(x='config', y='runtime_ms')) \
    + geom_boxplot() \
    + geom_hline(yintercept=0, color='white') \
    + coord_flip() \
    + xlab('') + ylab('program runtime [ms]') \
    + ggtitle("TTstar runtime")
    
print(plot)
plot.save('ttstar-runtime.pdf', width=5.6, height=2.6, units='in')

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit(
    X=ttstar.loc[:, ('inference', 'verification', 'normalisation')],
    y=ttstar['runtime_ms']
)
print('intercept =', model.intercept_)
print('coefficients =', model.coef_)

In [ ]:
smf.ols('runtime_ms ~ inference + verification + normalisation', data=ttstar).fit().summary()

In [ ]:
csc = xs.loc[xs['stage'] == 'csc']

smf.ols('runtime_ms ~ inference + verification + normalisation', data=csc).fit().summary()

In [ ]:
plot = ggplot(csc, aes(x='config', y='runtime_ms')) \
    + geom_boxplot() \
    + geom_hline(yintercept=0, color='white') \
    + xlab('') + ylab('code generation time [ms]') \
    + coord_flip() \
    + ggtitle("Code generation")
    
print(plot)
plot.save('csc-runtime.pdf', width=5.6, height=2.6, units='in')

In [ ]:
exec = xs.loc[(xs['stage'] == 'execution') & ~xs['normalisation'] & ~xs['verification']]

ggplot(exec.loc[exec['inference']], aes(x='input_size', y='runtime', color='config')) \
    + geom_point(alpha=0.3) \
    + ggtitle('Effect of compilation on erased programs')